# Machine Learning Algorithms

In [2]:
# enter into the correct directory 
! cd DATA/

In [1]:
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Notebook Specifics

pd.set_option("max_rows", 20)
np.set_printoptions(suppress = True)
import warnings
#Pretty Graphs

#from seaborn import set_style
#set_style("darkgrid")
#import seaborn as sns
#import warnings

In [ ]:
# This package will output the execution time of each cell. Pretty neat!

#warnings.filterwarnings('ignore')
#%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
#warnings.filterwarnings('default')
#%load_ext autotime

### We will start by reading into the training data and slicing the data into our features and Labels

In [3]:
data = pd.read_csv('training_clean.csv')

In [4]:
features = pd.DataFrame()
features = data[['tripduration', 'start station latitude', 'start station longitude', 'end station latitude', 'end station longitude', 'gender', 'age', 'day of week' ]]

In [5]:
labels = pd.DataFrame()
labels = data[['roundtrip']]

In [6]:
# in order to clear memory
del data

### Next we will read our testing data into test_set and results

In [7]:
file = pd.read_csv('testing_clean.csv')

In [8]:
test = pd.DataFrame()
test = file[['tripduration', 'start station latitude', 'start station longitude', 'end station latitude', 'end station longitude', 'gender', 'age', 'day of week']]

In [9]:
results = pd.DataFrame()
results = file['roundtrip']

In [10]:
# in order to clear memory
del file

In [67]:
from sklearn.decomposition import PCA 
from sklearn import preprocessing

#### Looking through some data 

In [ ]:
data.groupby('day of week')['start station name', 'end station name'].describe()

In [ ]:
features.head()
# features.groupby('roundtrip')['gender', 'age'].describe()

#### Now for applying PCA

In [68]:
pca = PCA(n_components = 2)
pca.fit(features)

MemoryError: 

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
print(pd.DataFrame(pca.components_,columns = features.columns, index = ['PC-1','PC-2']))

### Now lets see what happens when we scale the data before we apply PCA

In [ ]:
features = pd.DataFrame(preprocessing.scale(features),columns = features.columns)

MemoryError: 

In [ ]:
pca2 = PCA(n_components=3)
pca2.fit_transform(features_scaled)

In [ ]:
print("with scaling: ", pca2.explained_variance_ratio_)

In [ ]:
print(pd.DataFrame(pca2.components_,columns = features_scaled.columns, index = ['PC-1','PC-2', 'PC-3']))

In [ ]:
pca2.explained_variance_ratio_

In [ ]:
transformed_scaled_data = pca2.transform(features_scaled)

In [ ]:
first_pc=pca.components_[0]
second_pc=pca.components_[1]

##### For Plotting

In [ ]:
first_pc

In [ ]:
#for ii, jj in zip(transformed_scaled_data, data_scaled):
 #   plt.scatter(first_pc[0]*ii[0], first_pc[1]*ii[0], color = "r")
#   plt.scatter(second_pc[0]*ii[1], second_pc[1]*ii[1], color = "c")
  #  plt.scatter(jj[0], jj[1], color = "b")

#plt.xlabel("X-axis")
#plt.ylabel("Y-axis")
#plt.show()


In [ ]:
features.head(2)

### Now to TRY CART and Random Forests 

In [18]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz  

In [19]:
from sklearn import metrics   # for calculating the accuracy of our predictions

In [20]:
dtree = DecisionTreeClassifier(random_state=0, max_depth=2)

In [21]:
dtree.fit(features, labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [18]:
export_graphviz(dtree, out_file='classify.dot', feature_names=features.columns, filled=True, rounded=True,special_characters=True)   

In [19]:
from sklearn.externals import joblib # better than the pickle library 
                                    # more efficient on numpy arrays 
                                    # meant to write to disk so that we do not have to train our data set everytime
joblib.dump(dtree, 'Decision_Tree.pkl')   # stores $$$$ to filename.pkl which is a pickle file

#To retrieve the saved data, we need to use:
#$$$$ = joblib.load('filename.pkl')  

#   joblib.dump returns a list of filenames. Each individual numpy array
#   contained in the ``clf`` object is serialized as a separate file on the
#   filesystem. All files are required in the same folder when reloading the
#   model with joblib.load.

['Decision_Tree.pkl']

In [20]:
from graphviz import Source


In [21]:
def show_tree(out_file):
    dot = ''
    with open(out_file, 'r') as file:
        for line in file:
            dot += line
    dot = Source(dot)
    return dot

d = show_tree('classify.dot')
d.render(view=True)

'Source.gv.pdf'

In [22]:
x = dtree.predict_proba(test)

In [24]:
metrics.mean_absolute_error(results, x)

ValueError: y_true and y_pred have different number of output (1!=2)

In [28]:
print(x)

[[ 0.97738437  0.02261563]
 [ 1.          0.        ]
 [ 1.          0.        ]
 ..., 
 [ 1.          0.        ]
 [ 0.97738437  0.02261563]
 [ 0.8990085   0.1009915 ]]


In [57]:
x.shape

(3742900, 2)

In [58]:
results.shape

(3742900,)

In [59]:
result = pd.DataFrame(results, columns=['roundtrip'])

In [55]:
result.groupby('roundtrip')['roundtrip'].describe()

roundtrip       
0          count    3676761.0
           mean           0.0
           std            0.0
           min            0.0
           25%            0.0
           50%            0.0
           75%            0.0
           max            0.0
1          count      66139.0
           mean           1.0
           std            0.0
           min            1.0
           25%            1.0
           50%            1.0
           75%            1.0
           max            1.0
Name: roundtrip, dtype: float64

In [34]:
crap = pd.DataFrame(x, columns=['No Roundtip', 'Yes Roundtrip'])

In [63]:
crap[crap['Yes Roundtrip'] > 0.1]

,No Roundtip,Yes Roundtrip
6,0.899009,0.100991
8,0.899009,0.100991
32,0.899009,0.100991
33,0.899009,0.100991
48,0.899009,0.100991
49,0.899009,0.100991
50,0.899009,0.100991
57,0.899009,0.100991
109,0.795235,0.204765
129,0.795235,0.204765


In [39]:
crap.count()

No Roundtip      3742900
Yes Roundtrip    3742900
dtype: int64

In [65]:
crap[crap['Yes Roundtrip'] >= 0.21]

,No Roundtip,Yes Roundtrip


### From what I can tell Decision Trees couldn't predict higher than 21 % that a trip is a no trip
#### So either I suck at programming or decision trees was of no help at all
#### I would prefer to accept the former

### Maybe Try to implement PCA and then try Decision trees?

In [12]:
# Import the random forest package
from sklearn.ensemble import RandomForestClassifier 

### Ok Now To Try Random Forests

In [13]:
# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 100)

In [ ]:
# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(features, labels)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


In [ ]:
# Take the same decision trees and run it on the test data
output = forest.predict(test_data)

In [2]:
cd DATA/

/home/corpeus/Haris Data Project/CitiBike/DATA


SyntaxError: invalid syntax (<ipython-input-21-3673e296c2a5>, line 1)